### Import pakages.

In [362]:
import re
import csv
import json
import neo4j
import py2neo
from pprint import pprint
from py2neo import Database
from py2neo import Graph, Node, Relationship, NodeMatcher

In [363]:
# Connet neo4j database. 
HOST='localhost'
USER='neo4jproj'
PW='1234' # Your own password.

### Data preparation.

In [364]:
def get_val(data:dict,key:str) -> str:
    
    ''' Get values from dictionary, 
        if key not in dictionary return None '''
    
    if key not in data.keys():
        return None
    elif data[key]=='':
        return None
    else:
        return data[key]

#### Acquisition for all companies.

Created a csv file with company acquisition, year and amount.

In [365]:
# Dictiionary for acquisition. {company:{acquired_company:[year, amount]}...}
acquisition={}
with open('./info_box/Acquisition.csv') as Acquisition:
    acquisition_all = csv.DictReader(Acquisition)
    for row in acquisition_all:
        if row['Company'] in acquisition.keys():
            acquisition[row['Company']][row['Acquisition']]= [get_val(row,'Year'),get_val(row,'Amount')]
        else:
            acquisition[row['Company']]={}
            acquisition[row['Company']][row['Acquisition']] = [get_val(row,'Year'),get_val(row,'Amount')]
        

In [366]:
pprint(acquisition)

{'Alibaba': {'Alibaba Pictures': ['2014', ' $805 million'],
             'Amblin Partners': ['2016', None],
             'Ant Financial': ['2014', None],
             'South China Morning Post': ['2015', '$266 million '],
             'Vendio': ['2010', None]},
 'Amazon': {'Ring Inc.': ['2018', '$1000 million'],
            'Whole Foods Market': ['2017', '$13700 million'],
            'Zappos': ['2009', '$1200 million ']},
 'Walmart': {'Jet.com': ['2016', '$3000 million'],
             'Shoes.com': ['1993', '$70 million'],
             'Vudu': ['2010', '$100 million']},
 'eBay': {'Craigslist': ['2004', '$13.5 million'],
          'PayPal': ['2002', '$1500 million'],
          'Skype Technologies': ['2005', '$2600 million']}}


### NAICS Code: Create company_naics_code dictionary.

Note: Add company name to the company list in "scrape_naics.py".

- Youtube ==> Youku "can not find company" ;

- Paramount Pictures ==> Alibaba Pictures "can not find company" 

company_list = ['Alibaba', 'Walmart', 'eBay', 'Amazon','Youtube','Paramount Pictures','Costco','Kroger','Target','Vudu','Jet.com','Kmart']

In [367]:
# Unique naics codes.
naics=dict() 
# Company and 6 digits Naics Code, which we need create relation between code and company. 
company_code=dict() 

with open('./naics_scraped_all.csv') as file:
    data = csv.DictReader(file)
    for row in data:
        company = row['Company']
        code = row['NAICS_CODE']
        title = row['NAICS_TITLE']
        naics[code] = title
        if company not in company_code.keys():
             company_code[company]=code

In [368]:
company_code

{'Alibaba': '561990',
 'Walmart': '445110',
 'eBay': '561990',
 'Amazon': '443142',
 'Paramount Pictures': '512191',
 'Amblin': '334513',
 'Vendio': '442110',
 'Evening Post': '512191',
 'Vudu': '518210',
 'Jet.com': '237990',
 'Shoebuy.com': '454110',
 'Zappos': '448210',
 'Whole Foods Market': '722310',
 'Ring': '336412',
 'Skype Technologies': '611310',
 'PayPal': '551112',
 'Craigslist': '541860'}

In [369]:
# Change company names:
## 'Paramount Pictures' -> 'Alibaba Pictures'
## 'Ring' -> 'Ring Inc.' (Change to full name)
## 'Amblin' -> 'Amblin Partners' (Change to full name)
## 'Shoebuy.com' -> 'Shoes.com' (Shoebuy.com is former name)
## 'Evening Post' -> 'South China Morning Post'

company_code['Alibaba Pictures'] = company_code['Paramount Pictures']
company_code['Ring Inc.'] = company_code['Ring']
company_code['Amblin Partners'] = company_code['Amblin']
company_code['Shoes.com'] = company_code['Shoebuy.com']
company_code['South China Morning Post'] = company_code['Evening Post']

del company_code['Paramount Pictures']
del company_code['Ring']
del company_code['Amblin']
del company_code['Shoebuy.com']
del company_code['Evening Post']

# Add code for "Ant Financial"; use same code as PayPal
company_code.update( {'Ant Financial' : company_code['PayPal'] } )

In [370]:
# Extract unique root code.
root_code = [code for code in naics.keys() if len(code)==2 or re.match('\d+-\d+',code)]

# Code with 6 digits.
code6 = [code for code in naics.keys() if len(code)==6]
# Code with 5 digits.
code5 = [code for code in naics.keys() if len(code)==5 and not re.match('\d+-\d+',code)]
# Code with 4 digits.
code4 = [code for code in naics.keys() if len(code)==4]
# Code with 3 digits.
code3 = [code for code in naics.keys() if len(code)==3]



In [371]:
print(root_code)
print(code3)
print(code4)
print(code5)
print(code6,'\n')
pprint(company_code)

['56', '44-45', '51', '31-33', '23', '72', '61', '55', '54']
['561', '445', '443', '512', '334', '442', '518', '237', '454', '448', '722', '336', '611', '551', '541']
['5619', '4451', '4431', '5121', '3345', '4421', '5182', '2379', '4541', '4482', '7223', '3364', '6113', '5511', '5418']
['56199', '44511', '44314', '51219', '33451', '44211', '51821', '23799', '45411', '44821', '72231', '33641', '61131', '55111', '54186']
['561990', '445110', '443142', '512191', '334513', '442110', '518210', '237990', '454110', '448210', '722310', '336412', '611310', '551112', '541860'] 

{'Alibaba': '561990',
 'Alibaba Pictures': '512191',
 'Amazon': '443142',
 'Amblin Partners': '334513',
 'Ant Financial': '551112',
 'Craigslist': '541860',
 'Jet.com': '237990',
 'PayPal': '551112',
 'Ring Inc.': '336412',
 'Shoes.com': '454110',
 'Skype Technologies': '611310',
 'South China Morning Post': '512191',
 'Vendio': '442110',
 'Vudu': '518210',
 'Walmart': '445110',
 'Whole Foods Market': '722310',
 'Zappos

In [372]:
# naics

## Create graph. 
### Company Nodes and relationships

**NOTE** : I used the script in `'./scrapy_spider/scrapy_spider/spiders/wiki_spider_info.py'` to extracted the info box and saved in `info_box` folder.

In [373]:
def Create_Company_Node(json):
    
    '''Create company node'''
    
    company_node = Node('Company', 
                    name=get_val(json,'Title'),
                    organization_name=get_val(json,'Organization_name'),
                    founded=get_val(json,'Founded'),
                    founder=get_val(json,'Founder'),
                    industry=get_val(json,'industry'), 
                    products=get_val(json,'Products'), 
                    services=get_val(json,'Services'), 
                    revenue=get_val(json,'Revenue'), 
                    employees=get_val(json,'Number of employees'),
                    website=get_val(json,'Website'))
    
    company_node.__primarylabel__ = "Company"
    company_node.__primarykey__ = "name"
    
    return company_node

#### Alibaba

In [374]:
# Connect database.
g = Graph(host=HOST,user=USER,password=PW) 
tx = g.begin()

# Main company_Alibaba.
with open('./info_box/Alibaba.json') as file:
    data = json.load(file)
    for d in data:
        Alibaba = Create_Company_Node(d)         
        tx.create(Alibaba)

# Competitor_Alibaba. 
# ** Please chage the Amazon title in Competitor_Alibaba.json': "Amazon (Company)" --> "Amazon" ** 
with open('./info_box/Alibaba_Competitor.json') as file:
    data = json.load(file)
    for d in data:
        company_node = Create_Company_Node(d)
        relation = Relationship(Alibaba,'Competitor',company_node)
        tx.create(relation)
        
# Acquisition_Alibaba.
with open('./info_box/Alibaba_Acquisition.json') as file:
    data = json.load(file)
    for d in data:
        company_name = d['Title']
        company_node = Create_Company_Node(d)
        Y = acquisition['Alibaba'][company_name][0]
        A = acquisition['Alibaba'][company_name][1]
        relation = Relationship(Alibaba,'Acquired',company_node,year=Y,amount=A)
        tx.create(relation)        

tx.commit()

#### Walmart

In [375]:
# Connect database.
g = Graph(host=HOST,user=USER,password=PW) 
matcher = NodeMatcher(g)
tx = g.begin()

# Find Walmart node.
Walmart = matcher.match('Company',name='Walmart').first()
        
# Walmart compete with Amazon and eBay as well.         
Amazon = matcher.match('Company',name='Amazon').first()
eBay = matcher.match('Company',name='eBay').first()
tx.create(Relationship(Amazon,'Competitor',Walmart))
tx.create(Relationship(eBay,'Competitor',Walmart))

# Acquisition_Walmart. 
with open('./info_box/Walmart_Acquisition.json') as file:
    data = json.load(file)
    for d in data:
        company_name = d['Title']
        company_node = Create_Company_Node(d)
        Y = acquisition['Walmart'][company_name][0]
        A = acquisition['Walmart'][company_name][1]
        relation = Relationship(Walmart,'Acquired',company_node,year=Y,amount=A)
        tx.create(relation)
        
tx.commit()

#### Amazon

In [376]:
# Connect database.
g = Graph(host=HOST,user=USER,password=PW) 
matcher = NodeMatcher(g)
tx = g.begin()

# Find Walmart node.
Amazon = matcher.match('Company',name='Amazon').first()
        
# Amazon Acquisitions. 
with open('./info_box/Amazon_Acquisition.json') as acquisitions:
    data = json.load(acquisitions)
    for d in data:
        company_name = d['Title']
        company_node = Create_Company_Node(d)
        Y = acquisition['Amazon'][company_name][0]
        A = acquisition['Amazon'][company_name][1]
        relation = Relationship(Amazon,'Acquired',company_node,year=Y,amount=A)
        tx.create(relation)

tx.commit()

#### eBay

In [377]:
# Connect database.
g = Graph(host=HOST,user=USER,password=PW)
matcher = NodeMatcher(g)
tx = g.begin()

# Find Walmart node.
eBay = matcher.match('Company',name='eBay').first()

       
# Amazon Acquisitions. 
with open('./info_box/eBay_Acquisition.json') as acquisitions:
    data = json.load(acquisitions)
    for d in data:
        company_name = d['Title']
        company_node = Create_Company_Node(d)
        Y = acquisition['eBay'][company_name][0]
        A = acquisition['eBay'][company_name][1]
        relation = Relationship(eBay,'Acquired',company_node,year=Y,amount=A)
        tx.create(relation)

tx.commit()

In [378]:
# Add more relationships between companies.
# Connect database.
g = Graph(host=HOST,user=USER,password=PW)
matcher = NodeMatcher(g)
tx = g.begin()

shoes = matcher.match('Company',name='Shoes.com').first()
zappos = matcher.match('Company',name='Zappos').first()
tx.create(Relationship(shoes,'Competitor',zappos))

pp = matcher.match('Company',name='PayPal').first()
af = matcher.match('Company',name='Ant Financial').first()
tx.create(Relationship(pp,'Competitor',af))

amazon = matcher.match('Company',name='Amazon').first()
ebay = matcher.match('Company',name='eBay').first()
jet = matcher.match('Company',name='Jet.com').first()
tx.create(Relationship(jet,'Competitor',ebay))
tx.create(Relationship(jet,'Competitor',amazon))


tx.commit()

### NAICS code nodes and relationships.

In [379]:
def Create_Code_Node(code):
    
    '''Create NAICS Ccode node'''
    
    code_node = Node('NAICS', code=code, title=naics[code])
    
    code_node.__primarylabel__ = "NAICS"
    code_node.__primarykey__ = "code"
    
    return code_node
    

In [380]:
def Check_Code_Node(code):
    
    '''Check if NAICS Ccode node already exists'''
    
    node = matcher.match('NAICS', code=code)
    
    if node == None:
        new = Create_Code_Node(code)
        return new
    else:
        return node

#### Root Code. 

In [381]:
# Connect database.
g = Graph(host=HOST,user=USER,password=PW)
tx = g.begin()

# Create Root Code Node. (2-digits or d-d)
for root in root_code:
    root_node = Create_Code_Node(root)
    tx.create(root_node)
    
tx.commit()

#### 3-digit-code

In [382]:
# Connect database.
g = Graph(host=HOST,user=USER,password=PW)
tx = g.begin()
matcher = NodeMatcher(g)

# Create 3-digit-code Node.
for code in code3:
    node3 = Create_Code_Node(code)
    
    # Find the root code node.
    if code[:-1] in ['44','45']:
        node2 = matcher.match('NAICS', code='44-45').first()    
    elif code[:-1] in ['31','33']:
        node2 = matcher.match('NAICS', code='31-33').first()
    else:
        node2 = matcher.match('NAICS', code=code[:-1]).first()
        
    tx.create(Relationship(node3,'SubClassOf',node2))

tx.commit()

#### 4-digit-code

In [383]:
# Connect database
g = Graph(host=HOST,user=USER,password=PW)
tx = g.begin()
matcher = NodeMatcher(g)

# Create 4-digit-code Node.
for code in code4:
    node4 = Create_Code_Node(code)
    node3 = matcher.match('NAICS', code=code[:-1]).first()
    tx.create(Relationship(node4,'SubClassOf',node3))

tx.commit()

#### 5-digit-code

In [384]:
# Connect database.
g = Graph(host=HOST,user=USER,password=PW)
tx = g.begin()
matcher = NodeMatcher(g)

# Create 5-digit-code Node.
for code in code5:
    node5 = Create_Code_Node(code)
    node4 = matcher.match('NAICS', code=code[:-1]).first()
    tx.create(Relationship(node5,'SubClassOf',node4))

tx.commit()

#### 6-digit-code

In [385]:
# Connect database.
g = Graph(host=HOST,user=USER,password=PW)
tx = g.begin()
matcher = NodeMatcher(g)

# Create 5-digit-code Node.
for code in code6:
    node6 = Create_Code_Node(code)
    node5 = matcher.match('NAICS', code=code[:-1]).first()
    tx.create(Relationship(node6,'SubClassOf',node5))

tx.commit()

#### Create relationships between company and 6-digit-code.

In [386]:
# Connect database.
g = Graph(host=HOST,user=USER,password=PW)
tx = g.begin()

# Create relationships.
matcher = NodeMatcher(g)
for company in company_code:
    CompanyNode = matcher.match('Company', name=company).first()
    CodeNode = matcher.match('NAICS', code=company_code[company]).first()
    tx.create(Relationship(CodeNode,'COMPANY',CompanyNode))
    
tx.commit()

# Create controversies graph insertion


In [387]:
def Create_Controversy_Node(issue):
    
    '''Create controversy node'''
    
    node = Node('Controversy', issue=issue)
    
    node.__primarylabel__ = "CONTROVERSY"
    node.__primarykey__ = "controversy"
    
    return node
    

In [388]:
# table of contents particularly controversies
controversy_dict = {}
    
with open('./table_of_content.txt') as file:
    data = json.load(file)

    #mapping between data from toc and standarize
    data['Alibaba'] = data.pop('Alibaba_Group')
    data['eBay'] = data.pop('EBay')
    data['Amazon'] = data.pop('Amazon_(company)')

    for key,val in data.items():
        controversy_field = [x for x in list(val.keys()) if "controve" in x.lower()][0]
        controversy_dict[key] = val[controversy_field]
    print(controversy_dict)
    


{'Walmart': ['2010s crime problem'], 'Alibaba': ['Gold Supplier membership', 'Uranium sales', 'Counterfeit items and scams', 'Class action on IPO'], 'eBay': ['2014 security breach'], 'Amazon': ['Environmental impact', 'Selling counterfeit items', 'Sales and use taxes', 'Income taxes', 'Comments by Donald Trump and Bernie Sanders', 'Working conditions', 'Conflict of interest with the CIA and DOD', 'Seattle head tax and houselessness services', 'Nashville Operations Center of Excellence', 'Facial recognition technology and law enforcement']}


In [389]:
# Connect database.
g = Graph(host=HOST,user=USER,password=PW)
tx = g.begin()

# Create relationships.
matcher = NodeMatcher(g)
for company, controversy_list in controversy_dict.items():
    CompanyNode = matcher.match('Company', name=company).first()
    for controversy in controversy_list:
        controversy_node = Create_Controversy_Node(controversy)
        tx.create(Relationship(CompanyNode,'HasControversy',controversy_node ))
tx.commit()

# Add in areas competing in from info box

In [393]:
def Get_Node_or_Create(name, primary_label, primary_key, **kwargs):
    
    '''Create  node'''
    node = matcher.match(name, **kwargs).first()

    if node == None:
        node = Node(name, **kwargs )

        node.__primarylabel__ = primary_label
        node.__primarykey__ = primary_key

        return node
    else:
        return node

In [394]:
# Connect database.
g = Graph(host=HOST,user=USER,password=PW)


#as part of entity resolution, a simple way (no dedupe/nlp library) is to just have a mapping from company 
#to company as it is currently in neo4j 
company_mapping = {
    "Alibaba Group": "Alibaba",
    "Amazon (company)": "Amazon",
    "eBay": "eBay",
    "Walmart": "Walmart"
}


area_competing_in_map = {
    "Walmart": "Products",
    "Amazon": "Industry",
    "Alibaba": "Products",
    "eBay": "Services"
}
with open('./info_box/all_info_box.json') as all_info_box:
    info_box_data = json.load(all_info_box)

    for company in info_box_data:
        tx = g.begin()
        company_name = company_mapping[company['Title']]
        area_competing_in_key = area_competing_in_map.get(company_name, None)
        if (area_competing_in_key == None):
            area_competing_in_key = area_competing_in_map.get("Products")
        areas_competing_in = company[area_competing_in_key]

        CompanyNode = matcher.match('Company', name=company_name).first()
        for area in areas_competing_in.split(","):
            node = Get_Node_or_Create('Product_Industry_Name', "PRODUCT_INDUSTRY_NAME", "area_name", product_industry_name=area.strip())
            tx.create(Relationship(CompanyNode,'competes_in',node ))
        tx.commit()


        
    